In [1]:
import sys, os, pdb
import pandas as pd
import numpy as np
from datetime import date, timedelta
from dotenv import load_dotenv

load_dotenv()

from obility.agent import Agent
from obility.model import IndicatorList, KLine
from dichaos.agents.indexor.porfolio import Portfolio
from dichaos.kdutils import kd_logger

TA-Lib is not available: No module named 'talib'


In [2]:
symbol = 'RB'

In [3]:
### 构建模拟数据
n = 30
start_date = "2024-05-01"
trade_time = pd.date_range(start=start_date, periods=n, freq='B')
# 模拟价格数据
np.random.seed(42)
price = np.cumsum(np.random.randn(n)) + 100  # 模拟价格走势

open_price = price + np.random.randn(n) * 0.5
close_price = price + np.random.randn(n) * 0.5
high_price = np.maximum(open_price, close_price) + np.random.rand(n)
low_price = np.minimum(open_price, close_price) - np.random.rand(n)

# 模拟成交量和成交额
volume = np.random.randint(1000, 10000, size=n)
amount = volume * close_price

# 计算收益率（对数收益率）
returns = np.log(close_price / np.roll(close_price, 1))
returns[0] = 0  # 第一行无法计算对数收益率

# 构建 DataFrame
total_data = pd.DataFrame({
    'trade_time': trade_time,
    'open': open_price.round(2),
    'high': high_price.round(2),
    'low': low_price.round(2),
    'close': close_price.round(2),
    'volume': volume,
    'amount': amount.round(2),
    'return': returns.round(4)
})
total_data['code'] = symbol
total_data = total_data.set_index(['trade_time', 'code'])
total_data.head()

,,open,high,low,close,volume,amount,return
trade_time,code,,,,,,,
2024-05-01,RB,100.20,101.15,100.14,100.26,3385,339370.38,0.0000
2024-05-02,RB,101.28,101.82,99.99,100.27,5736,575123.60,0.0001
2024-05-03,RB,101.00,101.81,99.54,100.45,2802,281469.22,0.0019
2024-05-06,RB,102.00,102.90,101.69,101.93,9155,933178.90,0.0146
2024-05-07,RB,102.71,103.02,102.56,102.70,9120,936635.65,0.0075


In [4]:
### 创建 Agent
agent = Agent.from_config(path=os.path.join(Agent.name))

06-11 06:58 - Creating LLM service: openai model: deepseek-chat
06-11 06:58 - Creating vector service: fassis
06-11 06:58 - Creating embedding service: openai model: bge-m3
06-11 06:58 - Creating vector service: fassis
06-11 06:58 - Creating embedding service: openai model: bge-m3
06-11 06:58 - Creating vector service: fassis
06-11 06:58 - Creating embedding service: openai model: bge-m3
06-11 06:58 - Creating vector service: fassis
06-11 06:58 - Creating embedding service: openai model: bge-m3


In [5]:
### 计算策略依赖的技术指标
rsi_df = agent.calculate_rsi(total_data)

macd_df = agent.calculate_macd(total_data)


macd_df.tail()


,,macd,signal,hist
trade_time,code,,,
2024-06-05,RB,-1.4107,-0.8750,-0.5357
2024-06-06,RB,-1.5090,-1.0018,-0.5072
2024-06-07,RB,-1.5624,-1.1139,-0.4485
2024-06-10,RB,-1.6686,-1.2249,-0.4438
2024-06-11,RB,-1.7145,-1.3228,-0.3917


In [6]:
## 用于训练奖励机制
portfolio = Portfolio(symbol=symbol, lookback_window_size=0)

In [7]:
#### 首次训练
trade_time = '2024-06-07'

In [8]:
### K线数据
kline = KLine(date=trade_time,
                      symbol=symbol,
                      open=total_data.loc[(trade_time, symbol), 'open'],
                      close=total_data.loc[(trade_time, symbol), 'close'],
                      high=total_data.loc[(trade_time, symbol), 'high'],
                      low=total_data.loc[(trade_time, symbol), 'low'],
                      volume=total_data.loc[(trade_time, symbol), 'volume'])

print(kline.format())

2024-06-07 K线数据:
开盘价: 95.09
收盘价: 95.41
最高价: 95.45
最低价: 94.92
成交量: 3557.0



In [9]:
### 技术指标管理类
indicator_list = IndicatorList(date=trade_time)
indicator_list.set_indicator(rsi=rsi_df.loc[trade_time],
                             macd=macd_df.loc[trade_time])

print(indicator_list.format())

2024-06-07 技术指标:
RSI14: 28.0967 
MACD: -1.5624 



In [10]:
### 存入记忆池
agent.handing_data(trade_time, symbol, indicator_list, kline)

06-11 06:58 - {'text': '{"date":"2024-06-07","symbol":"RB","index":"-1","indicator":{"rsi":{"name":"RSI14","id":"rsi","values":28.0967},"macd":{"name":"MACD","id":"macd","values":-1.5624},"date":"2024-06-07"},"kline":{"date":"2024-06-07","symbol":"RB","open":95.09,"close":95.41,"high":95.45,"low":94.92,"volume":3557.0}}', 'id': 0, 'important_score': 90.0, 'recency_score': 1.0, 'delta': 0, 'important_score_recency_compound_score': 1.9, 'access_counter': 0, 'date': '2024-06-07'}


In [11]:
### 获取记忆池中的数据
short_prompt, reflection_prompt = agent.query_records(trade_time, symbol)

06-11 06:58 - reflection RB not in universe


In [12]:
print(short_prompt)

短期记忆索引ID: S0
2024-06-07 技术指标:
RSI14: 28.0967 
MACD: -1.5624 
2024-06-07 K线数据:
开盘价: 95.09
收盘价: 95.41
最高价: 95.45
最低价: 94.92
成交量: 3557.0



In [13]:
### 获取收益率转化为动作
future_data = total_data.loc[trade_time]
future_data = future_data.to_dict(orient='records')[0]
actions = 1 if future_data['return'] > 0 else -1

In [14]:
### 自我总结
response = agent.generate_suggestion(
            date=date,
            symbol=symbol,
            short_prompt=short_prompt,
            reflection_prompt=reflection_prompt,
            future_data=future_data)

06-11 06:58 - 
【核心决策要素】(请综合评估，寻找强共振信号)
    1. 评估MACD状态： 检查金叉/死叉情况、柱状图能量、与零轴的相对位置以及是否存在背离。
    2. 检查RSI水平： 判断是否处于超买/超卖区、是否接近50中轴、以及是否存在背离。
    3. 寻找信号共振点： 当MACD发出买入（卖出）信号，同时RSI也处于有利位置（如从超卖区向上，或突破50）时，买入（卖出）的确定性更高。
    4. 警惕背离信号： 出现顶背离时，即使价格仍在上涨，也应警惕回调风险，考虑减仓或设置止损；出现底背离时，即使价格仍在下跌，也可开始关注潜在的反弹机会。
    5. 结合周期分析： 短期信号需服从长期趋势。可参考不同时间周期的MACD和RSI表现，以提高决策准确性。

以下是短期记忆:
短期记忆索引ID: S0
2024-06-07 技术指标:
RSI14: 28.0967 
MACD: -1.5624 
2024-06-07 K线数据:
开盘价: 95.09
收盘价: 95.41
最高价: 95.45
最低价: 94.92
成交量: 3557.0


以下是过去反思记忆:



观察到的金融市场事实：对于 $RB， 下一个交易日与当前交易日之间出现了上涨信号，涨跌幅为：0.0009

根据短期记忆总结解释为什么 $RB  出现了 上涨 0.0009 的原因？
你需要提供一个总结决策信息和引用了短期记忆,过去反思记忆 信息ID来支持你的总结。注意只需要根据信息如实总结，不需要任何建议信息.

1. 必须严格按照以下JSON格式返回,分析内容必须中文描述.
2. 分析内容中不要出现索引ID编号 如S70,M70,L70这样的编号
3. 过去反思记忆用于验证当前短期记忆的有效性和准确性

{
"short_memory_index": "S1,S2,S3",
"reflection_memory_index":"R1,R2,R3",
"summary_reason": "string"
}




06-11 06:58 - error:'dict' object has no attribute '__dict__'
06-11 06:58 - 
【核心决策要素】(请综合评估，寻找强共振信号)
    1. 评估MACD状态： 检查金叉/死叉情况、柱状图能量、与零轴的相对位置以及是否存在背离。
    2. 检查RSI水平： 判断是否处于超买/超卖区、是否接近50中轴、以及是否存在背离。
    3. 寻找信号共振点： 当MACD发出买入（卖出）信号，同时RSI也处于有利位置（如从超卖区向上，或突破50）时，买入（卖出）的确定性更高。
    4. 警惕背离信号： 出现顶背离时，即使价格仍在上涨，也应警惕回调风险，考虑减仓或设置止损；出现底背离时，即使价格仍在下跌，也可开始关注潜在的反弹机会。
    5. 结合周期分析： 短期信号需服从长期趋势。可参考不同时间周期的MACD和RSI表现，以提高决策准确性。

以下是短期记忆:
短期记忆索引ID: S0
2024-06-07 技术指标:
RSI14: 28.0967 
MACD: -1.5624 
2024-06-07 K线数据:
开盘价: 95.09
收盘价: 95.41
最高价: 95.45
最低价: 94.92
成交量: 3557.0


以下是过去反思记忆:



观察到的金融市场事实：对于 $RB， 下一个交易日与当前交易日之间出现了上涨信号，涨跌幅为：0.0009

根据短期记忆总结解释为什么 $RB  出现了 上涨 0.0009 的原因？
你需要提供一个总结决策信息和引用了短期记忆,过去反思记忆 信息ID来支持你的总结。注意只需要根据信息如实总结，不需要任何建议信息.

1. 必须严格按照以下JSON格式返回,分析内容必须中文描述.
2. 分析内容中不要出现索引ID编号 如S70,M70,L70这样的编号
3. 过去反思记忆用于验证当前短期记忆的有效性和准确性

{
"short_memory_index": "S1,S2,S3",
"reflection_memory_index":"R1,R2,R3",
"summary_reason": "string"
}


06-11 06:59 - error:'dict' object has no attribute '__dict__'
06

In [15]:
response.__dict__

AttributeError: 'dict' object has no attribute '__dict__'

In [17]:
### 更新市场信息
portfolio.update_market_info(
            cur_date=trade_time,
            market_price=total_data.loc[trade_time, symbol]['close'],
            rets=total_data.loc[trade_time, symbol]['return'])

In [20]:
portfolio.record_action(action={'direction': actions})
feedback = portfolio.feedback()
### 更新记忆
agent.update_memory(trade_date=trade_time,
                    symbol=symbol,
                    response=response,
                    feedback=feedback)

06-03 07:47 - {'text': '根据短期记忆中的技术指标分析，RSI14值为28.0967，处于超卖区域，表明市场可能过度卖出，存在反弹的可能性。MACD值为-1.5624，虽然为负值，但结合RSI的超卖状态，市场可能开始出现买入信号。此外，收盘价从95.09上涨至95.41，显示出一定的上涨动能。因此，$RB出现了0.0009的上涨。', 'id': 1, 'important_score': 80.0, 'recency_score': 1.0, 'delta': 0, 'important_score_recency_compound_score': 1.8, 'access_counter': 0, 'date': '2024-06-07'}
06-03 07:47 - Memory jump starts...
06-03 07:47 - Memory jump ends...


In [21]:
### 保存记忆点
###  记忆点保存路径
agent.save_checkpoint(path=os.path.join(os.environ['BASE_PATH'], 'memory', agent.name, trade_time))

FileExistsError: Path records/memory/obility/2024-06-07/indexor/obility already exists

In [29]:
## 再次训练
trade_time = '2024-06-10'

In [30]:
### K线数据
kline = KLine(date=trade_time,
                      symbol=symbol,
                      open=total_data.loc[(trade_time, symbol), 'open'],
                      close=total_data.loc[(trade_time, symbol), 'close'],
                      high=total_data.loc[(trade_time, symbol), 'high'],
                      low=total_data.loc[(trade_time, symbol), 'low'],
                      volume=total_data.loc[(trade_time, symbol), 'volume'])

print(kline.format())

2024-06-10 K线数据:
开盘价: 94.81
收盘价: 94.38
最高价: 95.42
最低价: 93.69
成交量: 6592.0



In [31]:
### 技术指标管理类
indicator_list = IndicatorList(date=trade_time)
indicator_list.set_indicator(rsi=rsi_df.loc[trade_time],
                             macd=macd_df.loc[trade_time])

print(indicator_list.format())

2024-06-10 技术指标:
RSI14: 33.7691 
MACD: -1.6686 



In [32]:
### 存入记忆池
agent.handing_data(trade_time, symbol, indicator_list, kline)

06-03 07:52 - {'text': '{"date":"2024-06-10","symbol":"RB","index":"-1","indicator":{"rsi":{"name":"RSI14","id":"rsi","values":33.7691},"macd":{"name":"MACD","id":"macd","values":-1.6686},"date":"2024-06-10"},"kline":{"date":"2024-06-10","symbol":"RB","open":94.81,"close":94.38,"high":95.42,"low":93.69,"volume":6592.0}}', 'id': 2, 'important_score': 50.0, 'recency_score': 1.0, 'delta': 0, 'important_score_recency_compound_score': 1.5, 'access_counter': 0, 'date': '2024-06-10'}


In [33]:
### 获取记忆池中的数据
short_prompt, reflection_prompt = agent.query_records(trade_time, symbol)

In [34]:
print(short_prompt)

短期记忆索引ID: S2
2024-06-10 技术指标:
RSI14: 33.7691 
MACD: -1.6686 
2024-06-10 K线数据:
开盘价: 94.81
收盘价: 94.38
最高价: 95.42
最低价: 93.69
成交量: 6592.0
短期记忆索引ID: S1
2024-06-10 技术指标:
RSI14: 33.7691 
MACD: -1.6686 
2024-06-10 K线数据:
开盘价: 94.81
收盘价: 94.38
最高价: 95.42
最低价: 93.69
成交量: 6592.0



In [35]:
print(reflection_prompt)

过去反思记忆索引ID:R1  内容:根据短期记忆中的技术指标分析，RSI14值为28.0967，处于超卖区域，表明市场可能过度卖出，存在反弹的可能性。MACD值为-1.5624，虽然为负值，但结合RSI的超卖状态，市场可能开始出现买入信号。此外，收盘价从95.09上涨至95.41，显示出一定的上涨动能。因此，$RB出现了0.0009的上涨。
过去反思记忆索引ID:R0  内容:根据短期记忆中的技术指标分析，RSI14值为28.0967，处于超卖区域，表明市场可能过度卖出，存在反弹的可能性。MACD值为-1.5624，虽然为负值，但结合RSI的超卖状态，市场可能开始出现买入信号。此外，收盘价从95.09上涨至95.41，显示出一定的上涨动能。因此，$RB出现了0.0009的上涨。



In [36]:
### 获取收益率转化为动作
future_data = total_data.loc[trade_time]
future_data = future_data.to_dict(orient='records')[0]
actions = 1 if future_data['return'] > 0 else -1

In [37]:
### 自我总结
response = agent.generate_suggestion(
            date=date,
            symbol=symbol,
            short_prompt=short_prompt,
            reflection_prompt=reflection_prompt,
            future_data=future_data)

06-03 07:52 - 
【核心决策要素】(请综合评估，寻找强共振信号)
    1. 评估MACD状态： 检查金叉/死叉情况、柱状图能量、与零轴的相对位置以及是否存在背离。
    2. 检查RSI水平： 判断是否处于超买/超卖区、是否接近50中轴、以及是否存在背离。
    3. 寻找信号共振点： 当MACD发出买入（卖出）信号，同时RSI也处于有利位置（如从超卖区向上，或突破50）时，买入（卖出）的确定性更高。
    4. 警惕背离信号： 出现顶背离时，即使价格仍在上涨，也应警惕回调风险，考虑减仓或设置止损；出现底背离时，即使价格仍在下跌，也可开始关注潜在的反弹机会。
    5. 结合周期分析： 短期信号需服从长期趋势。可参考不同时间周期的MACD和RSI表现，以提高决策准确性。

以下是短期记忆:
短期记忆索引ID: S2
2024-06-10 技术指标:
RSI14: 33.7691 
MACD: -1.6686 
2024-06-10 K线数据:
开盘价: 94.81
收盘价: 94.38
最高价: 95.42
最低价: 93.69
成交量: 6592.0
短期记忆索引ID: S1
2024-06-10 技术指标:
RSI14: 33.7691 
MACD: -1.6686 
2024-06-10 K线数据:
开盘价: 94.81
收盘价: 94.38
最高价: 95.42
最低价: 93.69
成交量: 6592.0


以下是过去反思记忆:
过去反思记忆索引ID:R1  内容:根据短期记忆中的技术指标分析，RSI14值为28.0967，处于超卖区域，表明市场可能过度卖出，存在反弹的可能性。MACD值为-1.5624，虽然为负值，但结合RSI的超卖状态，市场可能开始出现买入信号。此外，收盘价从95.09上涨至95.41，显示出一定的上涨动能。因此，$RB出现了0.0009的上涨。
过去反思记忆索引ID:R0  内容:根据短期记忆中的技术指标分析，RSI14值为28.0967，处于超卖区域，表明市场可能过度卖出，存在反弹的可能性。MACD值为-1.5624，虽然为负值，但结合RSI的超卖状态，市场可能开始出现买入信号。此外，收盘价从95.09上涨至95.41，显示出一定的上涨动能。因此，$RB出现了0.0009的上涨。



观察到的金融市场事实

In [38]:
response.__dict__

{'short_memory_index': 'S1,S2',
 'reflection_memory_index': 'R1,R0',
 'summary_reason': '根据短期记忆中的技术指标分析，RSI14值为33.7691，虽然未达到超卖区域，但接近较低水平，表明市场可能开始出现卖出压力。MACD值为-1.6686，显示市场动能向下。结合K线数据，收盘价从94.81下跌至94.38，且最高价和最低价均显示下跌趋势，成交量6592.0表明市场参与度一般。过去反思记忆中提到RSI14值处于超卖区域时市场可能出现反弹，但当前RSI14值未达超卖区域，且MACD显示负值，市场动能向下，因此$RB出现了下跌-0.0109。'}

In [39]:
### 更新市场信息
portfolio.update_market_info(
            cur_date=trade_time,
            market_price=total_data.loc[trade_time, symbol]['close'],
            rets=total_data.loc[trade_time, symbol]['return'])

In [40]:
portfolio.record_action(action={'direction': actions})
feedback = portfolio.feedback()
### 更新记忆
agent.update_memory(trade_date=trade_time,
                    symbol=symbol,
                    response=response,
                    feedback=feedback)

06-03 07:53 - {'text': '根据短期记忆中的技术指标分析，RSI14值为33.7691，虽然未达到超卖区域，但接近较低水平，表明市场可能开始出现卖出压力。MACD值为-1.6686，显示市场动能向下。结合K线数据，收盘价从94.81下跌至94.38，且最高价和最低价均显示下跌趋势，成交量6592.0表明市场参与度一般。过去反思记忆中提到RSI14值处于超卖区域时市场可能出现反弹，但当前RSI14值未达超卖区域，且MACD显示负值，市场动能向下，因此$RB出现了下跌-0.0109。', 'id': 2, 'important_score': 80.0, 'recency_score': 1.0, 'delta': 0, 'important_score_recency_compound_score': 1.8, 'access_counter': 0, 'date': '2024-06-10'}
06-03 07:53 - Memory jump starts...
06-03 07:53 - Memory jump ends...


In [41]:
### 保存记忆点
agent.save_checkpoint(path=os.path.join(os.environ['BASE_PATH'], 'memory', agent.name, trade_time))

FileExistsError: Path records/memory/obility/2024-06-10/indexor/obility already exists